In [16]:
import numpy as np
import pygrib
from multiprocessing.pool import ThreadPool

## combine everything

In [17]:
storage_path = './data'

dates = ['20230914']
ftimes = ['00'] #['00','06' ,'12', '18']
lat_range = [25.1, 50]  # define the coordinates of the area
lon_range = [131, 231]

In [18]:
def get_filenames(path, ftime):
    filenames = []

    for i in range(385):
        # After 120h the forcasts are every 3 hours.
        if i > 120 and i % 3 != 0:
            continue

        file_num = '{:03d}'.format(i)
        filename = f'{path}/gfswave.t{ftime}z.global.0p25.f{file_num}.grib2'

        filenames.append(filename)
    
    return filenames

In [20]:
def filter_dataset(dataset, mask, dim):
    """
    For every GRIB message in the provided dataset:
        - masks the array to only the values in the lat/lon range
        - reshape np array to desired dimensions 
    """
    return [np.reshape(data.values[mask], dim).data for data in dataset]

In [26]:
def get_mask_dim(filename):
    """
    Calculates the required mask and dimensions for the weather data when read from the GRIB messages.
    Since all files and messages have the same shape and coordinates the mask and dimensions can be reused.
    """
    file = pygrib.open(filename)
    lats, lons = file[1].latlons()
    masklat = (lats >= lat_range[0]) & (lats <= lat_range[1])
    masklon = (lons >= lon_range[0]) & (lons <= lon_range[1])
    mask = masklat & masklon
    nlats = masklat[:,0].sum()
    nlons = masklon[0,:].sum()
    file.close()
    return mask, (nlats, nlons)

In [ ]:
"""
Iterates through all GRIB files for the provided ftime and collects the needed data from the messages.
The data is then stacked into one numpy array
"""
for ftime in ftimes:
    
    for date in dates:
        lwindspeed = []
        lwinddirection = []
        lshww = []
        lshsw = []
        lpww = []
        lpsw = []
        ldww = []
        ldsw = []

        path = f'{storage_path}/{date}/{ftime}'
        filenames = get_filenames(path, ftime)

        # Get mask and dimensions for reshaping the raw data read from the GRIB files
        mask, dim = get_mask_dim(filenames[0])
        # Needed messages from the GRIB files
        fields = [1, 2, 8, 9, 12, 13, 16, 17]

        for filename in filenames:
            file = pygrib.open(filename)
            data = filter_dataset([file[x] for x in fields], mask, dim)

            file.close()
    
            lwindspeed.append(data[0])
            lwinddirection.append(data[1])
            lshww.append(data[2])
            lshsw.append(data[3])
            lpww.append(data[4])
            lpsw.append(data[5])
            ldww.append(data[6])
            ldsw.append(data[7])
            print('finished', filename)

        windspeed = np.stack(lwindspeed,axis = 2)
        winddirection = np.stack(lwinddirection,axis = 2)
        shww = np.stack(lshww,axis = 2)
        shsw = np.stack(lshsw, axis = 2)
        pww = np.stack(lpww, axis = 2)
        psw = np.stack(lpsw, axis = 2)
        dww = np.stack(ldww, axis = 2)
        dsw = np.stack(ldsw, axis = 2)
        all_weather = np.stack((windspeed, winddirection, shww,pww,dww,shsw,psw,dsw), axis = 3)
        np.save(f'25gfs{date}_{ftime}', all_weather.data)
        
print('Finished normally')    

In [9]:
def nan_interpolate(arr_3d):
    '''This function is used to interpolate the NaN values'''
    result=np.zeros_like(arr_3d)
    for i in range(arr_3d.shape[0]):
        for j in range(arr_3d.shape[1]):
            arr=arr_3d[i,j,:]
            # If all elements are nan then cannot conduct linear interpolation.
            if np.sum(np.isnan(arr))==arr.shape[0]:
                result[i,j,:]=0
            else:
                # If the first elemet is nan, then assign the value of its right nearest neighbor to it.
                if np.isnan(arr[0]):
                    arr[0]=arr[~np.isnan(arr)][0]
                # If the last element is nan, then assign the value of its left nearest neighbor to it.
                if np.isnan(arr[-1]):
                    arr[-1]=arr[~np.isnan(arr)][-1]
                # If the element is in the middle and its value is nan, do linear interpolation using neighbor values.
                for k in range(arr.shape[0]):
                    if np.isnan(arr[k]):
                        x=k
                        x1=x-1
                        x2=x+1
                        # Find left neighbor whose value is not nan.
                        while x1>=0:
                            if np.isnan(arr[x1]):
                                x1=x1-1
                            else:
                                y1=arr[x1]
                                break
                        # Find right neighbor whose value is not nan.
                        while x2<arr.shape[0]:
                            if np.isnan(arr[x2]):
                                x2=x2+1
                            else:
                                y2=arr[x2]
                                break
                        # Calculate the slope and intercept determined by the left and right neighbors.
                        t1 = timeframe2hour(x1)
                        diff=(y2-y1)/(timeframe2hour(x2)-t1)                        
                        y=y1+diff*(np.abs(t1-timeframe2hour(x)))
                        arr[x]=y
                result[i,j,:]=arr
    return result

In [8]:
def timeframe2hour(tf):
    '''The number for the hours is from 0 to 209, but the real time frame is from 0 to 384. 
    From hour 120 onwards, every three hours, there is one frame of data. 
    So that the time frame has to be converted to real hour for interpolation of the weather attributes.'''
    if tf > 120:
        tf = (tf-120)*3 + 120
    return tf

In [11]:
s = 5  # to decrease the spatial precision on the longitude dimension
date = dates[0]
for ftime in ftimes:
    all_weather = np.load(f'25gfs{date}_{ftime}.npy')[:, 100::s, :, :]   # about index 100, select 300 points from the original 400 to avoid the land area near Japan
    print(all_weather.shape)
    all_weather[all_weather==9999]=np.nan   # to replace the invalid 9999 into np.NaN
    tws = []
    for w in range(all_weather.shape[3]):
        tw = nan_interpolate(all_weather[:,:,:,w])
        tws.append(tw)
    all_weather = np.stack((tws), axis = 3)
    np.save(f'gfs_NS_{date}{ftime}', all_weather)

(100, 60, 209, 8)


In [5]:
w0206 = np.load(f'nonan_2023091400.npy')
w0206.shape

(100, 60, 209, 8)

In [6]:
w0206[0,0,0,:]

array([  8.31, 238.55,   0.86,   5.36, 203.75,   0.72,   9.44, 146.28])

In [ ]:
w0206

In [ ]:
# the forecast at points (50N, 156E) at Hour 0 in the 20230206's '00' forecast
# wind speed, wind dir, wh, wp, wave direction, swell height, swell period, swell direction